# 고급 RAG 실습 노트북 (PDF/텍스트/채팅/퓨샷/사전변환)

이 노트북은 다양한 **고급 RAG(Retrieval-Augmented Generation)** 기법을 실습할 수 있도록 구성되어 있습니다.

---
**주요 실습 내용**
- PDF/텍스트 문서 청크화 및 벡터화
- ChromaDB 기반 저장 및 검색
- 다양한 RetrievalQA 체인 기법 (stuff, map_reduce, refine)
- Chat history 기반 ConversationalRetrievalChain
- LangChain Expression Language(LCEL) 기반 체인 구현
- Few-shot Prompting(퓨샷) 적용
- 입력 사전 변환 체인(RunnableLambda 등) 활용

---

## 1. 패키지 설치 및 환경 변수 로딩

- 실습에 필요한 패키지를 설치하고, .env 파일에서 OpenAI API 키를 불러옵니다.

In [ ]:
%pip install -r ../requirements.txt

from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key, "OPENAI_API_KEY 환경변수가 필요합니다."

## 2. PDF/텍스트 문서 로딩 및 청크 분할

- 여러 PDF 파일을을 로드하여 청크 단위로 분할합니다.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

data_dir = "../data"
pdf_files = glob.glob(os.path.join(data_dir, "*.pdf"))

all_documents = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    docs = loader.load()
    all_documents.extend(docs)
    print(f"{os.path.basename(pdf_path)} 문서 {len(docs)}개 로드됨")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=250,
    separators=["\n", ".", " "]
)
chunks = text_splitter.split_documents(all_documents)
print(f"총 청크 개수: {len(chunks)}")
for chunk in chunks[:3]:
    print('-', chunk.page_content)

## 3. ChromaDB 벡터스토어 구축 및 저장

- 분할된 청크 데이터를 임베딩(벡터화)하여 ChromaDB에 저장합니다.
- persist_directory를 지정하면 DB가 파일로 저장되어 재사용이 가능합니다.

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large', openai_api_key=openai_api_key)
persist_dir = "../chroma"

vectorstore = Chroma.from_documents(chunks, embedding, persist_directory=persist_dir)

## 4. RetrievalQA 체인 기법 비교 (stuff, map_reduce, refine)

- RetrievalQA 체인은 검색된 문서들을 LLM에 어떻게 전달하고 답변을 생성할지 다양한 방식을 제공합니다.
- 각 체인 타입의 특징은 아래와 같습니다.

| 체인 타입      | 동작 방식 요약                                                                 | 장점/특징                                  |
| -------------- | ----------------------------------------------------------------------------- | ------------------------------------------ |
| **stuff**      | 모든 검색 문서를 한 번에 LLM에 입력하여 답변 생성                              | 문서가 적을 때 빠르고 간단, 답변이 일관됨   |
| **map_reduce** | 각 문서별로 LLM이 부분 답변(map) → 부분 답변을 다시 LLM에 입력해 최종 답변(reduce) | 문서가 많을 때 유리, 요약/통합에 강점       |
| **refine**     | 첫 문서로 초안 생성 후, 나머지 문서를 순차적으로 추가하며 답변을 점진적으로 보완 | 문서별로 점진적 보완, 긴 문서에 적합        |

- 아래 셀에서 각 체인 타입별로 같은 질문을 입력해 답변 결과와 차이를 직접 비교해볼 수 있습니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "휴가 관련된 내용 알려줘!"

for chain_type in ["stuff", "map_reduce", "refine"]:
    print(f"\n--- RetrievalQA 체인 타입: {chain_type} ---")
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type=chain_type,
        return_source_documents=False
    )
    result = chain.invoke({"query": query})
    print(result["result"])

## 5. ConversationalRetrievalChain (채팅 히스토리 기반)

- 사용자의 이전 질문/답변 히스토리를 반영하여 문맥에 맞는 답변을 생성합니다.
- 실제 챗봇 시나리오에 가까운 체험이 가능합니다.

In [ ]:
from langchain.chains import ConversationalRetrievalChain

conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

chat_history = []
questions = [
    "출산휴가 관련해서 알려줘!",
    "배우자가 하면?",
    "마일리지 제도는 어떻게 돼?"
]

for question in questions:
    result = conv_chain.invoke({"question": question, "chat_history": chat_history})
    print(f"Q: {question}")
    print("A:", result["answer"])
    chat_history.append((question, result["answer"]))

## 6. Few-shot Prompting(퓨샷) 적용

- LLM 프롬프트에 예시(샷)를 추가하여 원하는 답변 스타일을 유도할 수 있습니다.
- 아래 예시는 Q/A 형식의 샷을 추가하여, 답변의 일관성과 품질을 높입니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트에서 context와 query(질문) 모두 명시적으로 사용해야 하며,
# RetrievalQA의 입력 변수명과 프롬프트 변수명이 일치해야 합니다.
fewshot_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친근한 답변을 하는 한국어 비서야. 이모티콘도 활용해줘."),
    ("human", "휴가 제도 알려줘!"),
    ("ai", "네! 😊 휴가 제도는 연차, 출산휴가, 경조휴가 등 다양한 종류가 있어요. 궁금한 점을 더 물어봐주세요!"),
    ("human", "{question}"), # <-- 여기를 {question}으로 변경
    ("system", "참고 문서:\n{context}")
])

print("ChatPromptTemplate 정의 완료.")
print("-" * 30)

# --- 2. RetrievalQA 체인 정의 ---
# RetrievalQA에서 prompt를 사용할 때는 chain_type="stuff"로 명시하고,
# document_variable_name을 "context"로 지정해야 합니다.
qa_chain_fewshot = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": fewshot_prompt,
        "document_variable_name": "context" # 참고 문서 변수명
    }
)

print("RetrievalQA 체인 정의 완료.")
print("-" * 30)

# --- 3. 질의 실행 ---
# invoke 메서드에 전달하는 딕셔너리의 키는 ChatPromptTemplate에서 사용된 플레이스홀더 이름과 일치해야 합니다.
# {question}에 해당하는 값으로 "query"를 전달합니다.
# RetrievalQA는 내부적으로 "query"를 "question"으로 매핑하는 경우가 많습니다.
# 만약 여전히 문제가 발생하면 invoke의 키를 "question"으로 변경해 볼 수 있습니다.
result = qa_chain_fewshot.invoke({"query": "마일리지 제도 알려줘!"})
print("\n--- 결과 ---")
print(result["result"])


# 7. 입력 사전 변환 체인(RunnableLambda 등) 실습
RAG(Retrieval-Augmented Generation) 시스템에서 사용자의 원본 질문을 LLM에 전달하거나 검색기에 넘기기 전에, 전처리(Pre-processing) 과정을 거쳐 품질을 향상시킬 수 있습니다. RunnableLambda와 같은 LangChain의 유연한 컴포넌트들을 활용하면 이러한 전처리 로직을 체인 내에 쉽게 통합할 수 있습니다.

입력 사전 변환은 다음과 같은 목적으로 사용될 수 있습니다:

- 쿼리 재작성 (Query Rewriting): 사용자의 질문이 모호하거나, 대화 이력이 있는 경우, 또는 검색에 비효율적인 표현을 포함할 때, LLM을 사용하여 검색에 최적화된 새로운 쿼리로 변환합니다. 이는 벡터 DB 검색의 정확도를 크게 높일 수 있습니다.
- 키워드 추출: 질문에서 핵심 키워드만을 추출하여 검색에 활용합니다.
- 입력 포맷팅: 여러 종류의 입력값을 특정 형식에 맞춰 조합하거나 변환합니다.
- 불필요한 정보 제거: 질문 내의 감탄사, 불필요한 서론 등을 제거하여 핵심만 남깁니다.

아래 예시는 RunnableLambda와 LLM을 조합하여, 사용자의 원본 질문을 검색에 더 적합한 키워드 쿼리로 재작성한 후 이를 검색 및 답변 생성 체인에 전달하는 방법을 보여줍니다. 이는 RAG 시스템의 검색 정확도를 높이는 데 매우 효과적인 고급 기법입니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser # LLM 출력 파싱용

# --- 1. 쿼리 재작성 프롬프트 정의 ---
# 사용자의 질문을 벡터 데이터베이스에서 검색하기 가장 좋은 키워드 쿼리로 변환하도록 LLM에게 지시합니다.
query_rewriter_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자 질문을 벡터 데이터베이스에서 검색하기 가장 좋은 키워드 쿼리로 변환하는 전문가야. 질문의 핵심을 파악하여 간결하고 명확한 검색어만 추출해줘. 예를 들어, '휴가 제도에 대해 자세히 알려주세요.' -> '휴가 제도', '회사 복지에 대해 알려줘' -> '회사 복지'"),
    ("human", "{original_query}") # 원본 질문을 받을 플레이스홀더
])

# --- 2. 쿼리 재작성 LLM 체인 생성 ---
# LLM을 사용하여 original_query를 검색 쿼리로 변환합니다.
query_rewriter_chain = query_rewriter_prompt | llm | StrOutputParser()

# --- 3. 쿼리 재작성 함수 정의 (RunnableLambda에 사용될) ---
def get_rewritten_query_for_chain(inputs):
    """
    주어진 입력을 기반으로 쿼리 재작성 체인을 실행하고 결과를 반환합니다.
    qa_chain_fewshot이 기대하는 'question' 키로 반환합니다.
    """
    # query_rewriter_chain은 {original_query}를 입력으로 받으므로, inputs["query"]를 전달합니다.
    rewritten_q = query_rewriter_chain.invoke({"original_query": inputs["query"]})
    print(f"DEBUG: 원본 질문: '{inputs['query']}' -> 재작성된 쿼리: '{rewritten_q.strip()}'") # 디버깅 출력
    return {"question": rewritten_q.strip()} # RetrievalQA가 기대하는 "question" 키로 반환

# --- 4. 전체 파이프라인 구축: 쿼리 재작성 -> RAG 체인 ---
# 입력: {"query": "사용자 질문"}
# 출력: {"result": "답변"}
qa_chain_preprocess = (
    RunnablePassthrough.assign( # 입력{"query"}를 다음 단계로 그대로 전달하면서
        # 'question'이라는 새로운 키로 쿼리 재작성 결과를 추가합니다.
        # 이 'question'은 qa_chain_fewshot의 입력으로 사용됩니다.
        question=RunnableLambda(get_rewritten_query_for_chain)
    )
    | qa_chain_fewshot # 재작성된 쿼리가 포함된 입력이 qa_chain_fewshot으로 전달됨
)

# --- 5. 질의 실행 ---
result = qa_chain_preprocess.invoke({"query": "마일리지 제도 알려줘!"})
print(result["result"])

## 8. LCEL 기반 체인 구성 (LangChain Expression Language)

- LCEL을 활용해 체인 조합을 자유롭게 실험할 수 있습니다.
- 아래 예시는 검색 → 프롬프트 조합 → 답변 생성을 모두 LCEL로 구현합니다.

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

qa_prompt = PromptTemplate.from_template(
    """
    당신은 AK아이에스의 친근한 챗봇 비서입니다. 😊
    제공된 '참고 문서'에 기반하여 질문에 답변합니다.
    다음 지시사항을 반드시 준수하세요:
    1. 이모티콘을 적절히 활용하여 친근하고 부드러운 말투로 답변하세요.
    2. '참고 문서'에 없는 내용이나 관련 없는 질문에는 '답변 할 수 없습니다.'라고 명확하게 답변하세요.
    3. 답변은 가능한 한 간결하고 명확하게 작성하세요.
    
    참고 문서:
    {context}

    질문: {question}
    답변:
    """
    )

combine_documents_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=qa_prompt,
    document_variable_name="context"
)

def input_mapper(inputs):
    return {
        "context": retriever.get_relevant_documents(inputs["question"]),
        "question": inputs["question"]
    }

rag_chain = RunnableLambda(input_mapper) | combine_documents_chain

try:
    response = rag_chain.invoke({"question": "마일리지는 누가 받을 수 있나요?"})

    print(response["answer"] if isinstance(response, dict) and "answer" in response else response)
except Exception as e:
    print("RAG 체인 실행 중 오류:", e)